In [1]:
import mlflow
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df_trimed = pd.read_csv("../Data/dataset_final.csv")
x_train, x_test, y_train, y_test = train_test_split(df_trimed.loc[:,df_trimed.columns != 'TARGET'], df_trimed['TARGET'], test_size = 0.3, random_state = 50)

In [3]:
def train(in_alpha, in_l1_ratio, x_test, y_test, x_train, y_train):
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet

    import mlflow
    import mlflow.sklearn
    
    import logging
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    np.random.seed(40)

    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(x_train, y_train)

        # Evaluate Metrics
        predicted_qualities = lr.predict(x_test)
        (rmse, mae, r2) = eval_metrics(y_test, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [11]:
mlflow.set_tracking_uri("http://localhost:5000")

In [12]:
run = train(0.5, 0.5, x_test, y_test, x_train, y_train)
run

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.27842508330150295
  MAE: 0.15617360468733568
  R2: 0.012792341214732716
